In [19]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from catboost import CatBoostClassifier

**Считываем данные**

In [2]:
train_df = pd.read_csv('../../data/flight_delays_train.csv')

In [3]:
train_df.head()

Month DayofMonth DayOfWeek  DepTime UniqueCarrier Origin Dest  Distance  \
0   c-8       c-21       c-7     1934            AA    ATL  DFW       732   
1   c-4       c-20       c-3     1548            US    PIT  MCO       834   
2   c-9        c-2       c-5     1422            XE    RDU  CLE       416   
3  c-11       c-25       c-6     1015            OO    DEN  MEM       872   
4  c-10        c-7       c-6     1828            WN    MDW  OMA       423   

  dep_delayed_15min  
0                 N  
1                 N  
2                 N  
3                 N  
4                 Y

In [4]:
test_df = pd.read_csv('../../data/flight_delays_test.csv')

In [5]:
test_df.head()

Month DayofMonth DayOfWeek  DepTime UniqueCarrier Origin Dest  Distance
0   c-7       c-25       c-3      615            YV    MRY  PHX       598
1   c-4       c-17       c-2      739            WN    LAS  HOU      1235
2  c-12        c-2       c-7      651            MQ    GSP  ORD       577
3   c-3       c-25       c-7     1614            WN    BWI  MHT       377
4   c-6        c-6       c-3     1505            UA    ORD  STL       258

**Создаем всего один признак – маршрут**

In [6]:
train_df['flight'] = train_df['Origin'] + '-->' + train_df['Dest']
test_df['flight'] = test_df['Origin'] + '-->' + test_df['Dest']

**Запоминаем номера категориальных признаков**

In [7]:
categ_feat_idx = np.where(train_df.drop('dep_delayed_15min', axis=1).dtypes == 'object')[0]
categ_feat_idx

array([0, 1, 2, 4, 5, 6, 8])

**Выделяем отложенную выборку**

In [8]:
X_train = train_df.drop('dep_delayed_15min', axis=1).values
y_train = train_df['dep_delayed_15min'].map({'Y': 1, 'N': 0}).values
X_test = test_df.values

In [9]:
X_train_part, X_valid, y_train_part, y_valid = train_test_split(X_train, y_train, 
                                                                test_size=0.3, 
                                                                random_state=17)

In [10]:
ctb = CatBoostClassifier(random_seed=17)

**Обучаем Catboost без настройки параметров, передав только индексы категориальных признаков.**

In [11]:
%%time
ctb.fit(X_train_part, y_train_part,
        cat_features=categ_feat_idx)

CPU times: user 5min 16s, sys: 43.3 s, total: 5min 59s
Wall time: 59.1 s


In [12]:
ctb_valid_pred = ctb.predict_proba(X_valid)[:, 1]

**Получаем почти 0.75 ROC AUC на отложенной выборке.**

In [13]:
roc_auc_score(y_valid, ctb_valid_pred)

0.74978752558569872

**Обучаем на всей выборке, делаем прогноз на тестовой, в соревновании получается результат 0.73008.**

In [14]:
%%time
ctb.fit(X_train, y_train,
        cat_features=categ_feat_idx)

CPU times: user 6min 26s, sys: 47.9 s, total: 7min 14s
Wall time: 1min 12s


In [15]:
ctb_test_pred = ctb.predict_proba(X_test)[:, 1]

In [16]:
sample_sub = pd.read_csv('sample_submission.csv', index_col='id')
sample_sub['dep_delayed_15min'] = ctb_test_pred
sample_sub.to_csv('ctb_pred.csv')

In [17]:
sample_sub.head()

dep_delayed_15min
id                   
0            0.038922
1            0.059309
2            0.042454
3            0.244527
4            0.291769